# Deploy and inference Jina Reranker M0 with Azure Virtual machine

This notebook demonstrates how to deploy a Jina Reranker M0 model-powered [Azure Virtual machine](https://azuremarketplace.microsoft.com/en-us/marketplace/apps/jinaai.jina-reranker-m0?tab=Overview) and perform inference with the application within the virtual machine.

## Deploy the virtual machine

To deploy the virtual machine, start by consulting the [official deployment guide](https://learn.microsoft.com/en-us/azure/virtual-machines/windows/quick-create-portal). This document provides comprehensive steps for the deployment process.

It's worth mentioning that in the Basics tab of the VM setup, you will need to provide several details about the VM. 

You can customize the VM size used, and for certain sizes, you might need to adjust the allowed quota to ensure access. It is recommended to use the [Standard_NC4as_T4_v3](https://learn.microsoft.com/en-us/azure/virtual-machines/nct4-v3-series) VM. This VM features up to 1 NVIDIA T4 GPU with 16 GB of memory.

For the other tabs, you can leave most settings as default or adjust them to fit your needs.

<img src="images/deploy_reranking_m0.png" width="50%" height="50%">

Once the deployment of the VM is complete, proceed to the resource group created for your deployment to verify the resources that have been established. 

Within the resource group, you'll find the public IP through which you can access your application within the VM.

Please note that the application within the VM will be unavailable immediately after deployment due to necessary model loading process **We recommend waiting at least 2 minutes before using the application.**

# Perform inference with the application

The Python example below demonstrates how to perform real-time inference using the public IP address of the deployed virtual machine.

In [ ]:
import json

import requests
import numpy as np
from PIL import Image
from io import BytesIO
import base64

random_image = np.random.randint(0, 256, (224, 224, 3), dtype=np.uint8)
pil_image = Image.fromarray(random_image)
buffered = BytesIO()
pil_image.save(buffered, format="PNG")
image_bytes = buffered.getvalue()
image_bytes_str = base64.b64encode(image_bytes).decode("utf-8")


def invoke_endpoint():
    url = "http://<Insert here your public IP address>/invocations"  # For example: http://20.84.48.180/invocations
    headers = {"Content-Type": "application/json"}
    json_data = {"data":[{
        "query": {"text": "where is the dog?"},
        "documents": [
            {
                "text": "I am a text"
            },
            {
                "text": "The dog is right here!"
            },
            {
                "image": "https://cdn.britannica.com/68/170868-050-8DDE8263/Golden-Gate-Bridge-San-Francisco.jpg"
            },
            {
                'bytes': image_bytes_str,
            },
        ]
    }]}

    response = requests.post(url, headers=headers, data=json.dumps(json_data))
    print(response.json())


invoke_endpoint()